In [1]:
!pip install camelot-py[cv] img2table openpyxl pandas pillow pytesseract


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.1 MB/s eta 0:00:00


In [8]:
%pip install img2table

import camelot
import os
from img2table.document import Image as Img2TableImage
from img2table.ocr import TesseractOCR
from PIL import Image
import pandas as pd
from openpyxl import Workbook

def extract_tables_from_pdf(pdf_path):
    # Extract tables from PDF using Camelot
    tables = camelot.read_pdf(pdf_path, pages='all')
    dfs = [table.df for table in tables]
    return dfs

def extract_tables_from_image(image_path):
    # Extract tables from an image using img2table and Tesseract
    img = Img2TableImage(src=image_path)
    ocr = TesseractOCR(lang="eng")
    tables = img.extract_tables(ocr=ocr)
    dfs = []
    for table in tables:
        # Each table is a Table object with the "df" attribute as DataFrame
        dfs.append(table.df)
    return dfs

def save_dfs_to_excel(dfs, output_excel, sheet_base_name="Table"):
    # Save multiple DataFrames to Excel, each to a separate sheet
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        for i, df in enumerate(dfs):
            df.to_excel(writer, sheet_name=f"{sheet_base_name}_{i+1}", index=False)

def main(pdf_files, image_files, output_excel="all_tables_combined.xlsx"):
    all_tables = []
    # Extract from PDFs
    for pdf_file in pdf_files:
        print(f"Extracting tables from PDF: {pdf_file}")
        pdf_tables = extract_tables_from_pdf(pdf_file)
        all_tables.extend(pdf_tables)
    # Extract from images
    for img_file in image_files:
        print(f"Extracting tables from image: {img_file}")
        img_tables = extract_tables_from_image(img_file)
        all_tables.extend(img_tables)
    # Save all tables to a single Excel
    if all_tables:
        save_dfs_to_excel(all_tables, output_excel)
        print(f"All tables saved into {output_excel}")
    else:
        print("No tables found in provided files.")

if __name__ == "__main__":
    # Example file lists
    pdf_files = []   # Your PDF files here
    image_files = ["/content/Revoult Bank statement_page-0001.jpg"]   # Your image files here - Added empty list
    main(pdf_files, image_files, output_excel="tables_from_pdf_and_images.xlsx")

Extracting tables from image: /content/Revoult Bank statement_page-0001.jpg
No tables found in provided files.
